In [55]:
import requests as req , re , pandas as pd

In [56]:
def connect(source):
    url = 'https://t.me/s/' + source
    request = req.get(url)
    html = request.text
    print('connected to' , source)
    return html

In [57]:
def targets(html , source):
    regex = re.compile(r'href=.https://t.me/[^/]\w+')
    data = []
    for target in regex.findall(html):
        target = target.rsplit('/' , 1)[1]
        if target.lower() != source.lower():
            data.append([source, target])
    edge_df = pd.DataFrame(data , columns=['source', 'target'])
    print('got targets from' , source)
    return edge_df

In [58]:
def subs (html , source):
    regex = re.compile(r'class=.counter_value.>[^<]+')
    try:
        string = regex.findall(html)[0]
        number = convert_str_to_number(string.rsplit('>' , 1)[1])
        print('got ' , number ,  'subs from ' , source)
        return number
    except:
        print('cant get subs for' , source)
    return

note: connect , targets and subs functions can be edited per social network

In [59]:
def convert_str_to_number(x):
    total_stars = 0
    num_map = {'K':1000, 'M':1000000, 'B':1000000000}
    if x.isdigit():
        total_stars = int(x)
    else:
        if len(x) > 1:
            total_stars = float(x[:-1]) * num_map.get(x[-1].upper(), 1)
    return int(total_stars)

In [60]:
def finals(source):
    html = connect(source)
    size = subs(html , source)
    edge_df = targets(html , source)
    edge_df['source_node_size'] = size
    edge_df['edge_size'] = edge_df.groupby(['target'])['source'].transform('count')
    edge_df = edge_df.drop_duplicates(subset=['target'])
    print('passed finals for' , source)
    print('edge_df in finals' , edge_df)
    return edge_df

In [61]:
def loop(df , i , start_channel_name):
    final_df  = pd.DataFrame()
    print('final df in loop' , df)
    targets = df['target']
    print('targets in loop' , targets)
    for target in targets:
        edge_df = finals(target)
        final_df = final_df.append(edge_df)
        print('current length = ' , len(final_df))
    final_df.to_csv(str(i) + ' iteration for ' + start_channel_name + '.csv')
    i += 1
    print('iteration' , i)
    loop(final_df , i , start_channel_name)
    return

In [62]:
#channel_name = input()
channel_name = 'rusvesnasu'
loop(finals(channel_name) , 0 , channel_name)

connected to rusvesnasu
got subs from rusvesnasu 7920
got targets from rusvesnasu
edge_df from targets Empty DataFrame
Columns: [source, target]
Index: []
passed finals for rusvesnasu
edge_df in finals Empty DataFrame
Columns: [source, target, source_node_size, edge_size]
Index: []
final df in loop Empty DataFrame
Columns: [source, target, source_node_size, edge_size]
Index: []
targets in loop Series([], Name: target, dtype: object)
iteration 1
final df in loop Empty DataFrame
Columns: []
Index: []


KeyError: 'target'